In [1]:
import pandas as pd
from math import log

In [2]:
# read in dataframe
df = pd.read_csv("Task 3 and 4_Loan_Data.csv")

In [3]:
# we are going to calculate the prefix sums of the number of records and the number of defaults for all possible scores according to the dataframe
prefix_records = [0] * (552) # 551 scores from first index onwards, index 0 is 0 to compute the first prefix sum
prefix_defaults = [0] * (552)

In [4]:
for idx in range(1, 552):
    cur_score = idx + 299 # 299 acts as a buffer -> scores 300 to 850
    prefix_records[idx] = len(df[df["fico_score"] == cur_score]) + prefix_records[idx-1]
    prefix_defaults[idx] = len(df[(df["fico_score"] == cur_score) & (df["default"] == 1)]) + prefix_defaults[idx-1]

In [5]:
# store the LL (Log-Likelihood) for all possible buckets
LL_mat = [[0]*552 for _ in range(552)]
for start in range(1, 552):
    for end in range(start, 552):
        n = prefix_records[end] - prefix_records[start-1]
        k = prefix_defaults[end] - prefix_defaults[start-1]
        if k == 0 or k == n: # according to the LL equation, the equation will give 0 in these cases
            LL_mat[start][end] = 0
        else:
            LL_mat[start][end] = k * log(k / n) + (n - k) * log(1 - (k/n))

In [6]:
### Method 1 (brute-force)
# try 4 buckets so there are 3 dividers -> O(n^3) solution but checks all possibilites
res = []
maximum = float('-inf')

# each divider (e.g. first = 1 means the divider will be placed after the 1st possible FICO score which is 300 so the first bucket will then be [300].)
for first in range(1, 548):
    for second in range(first+1, 549):
        for third in range(second+1, 550):
            LL = LL_mat[1][first] + LL_mat[first+1][second] + LL_mat[second+1][third] + LL_mat[third+1][551]
            if LL > maximum:
                maximum = LL
                res = [first, second, third]

In [7]:
print(res)

[253, 311, 350]


In [8]:
print(maximum) # the maximum log likelihood due to the bucketing

-4276.732521597704


In [9]:
print("The 4 optimal buckets are:")
print(f"Score 1 (Lowest Credit Band): 300 to {res[0]+299}")
print(f"Score 2 (Second Lowest Credit Band): {res[0]+300} to {res[1]+299}")
print(f"Score 3 (Second Highest Credit Band): {res[1]+300} to {res[2]+299}")
print(f"Score 4 (Highest Credit Band): {res[2]+300} to 850")

The 4 optimal buckets are:
Score 1 (Lowest Credit Band): 300 to 552
Score 2 (Second Lowest Credit Band): 553 to 610
Score 3 (Second Highest Credit Band): 611 to 649
Score 4 (Highest Credit Band): 650 to 850


In [10]:
### Method 2 (Adapted from suggested answer) -> Dynamic Programming
iter_num = 4
dp = [[[float("-inf"), 1] for i in range(552)] for j in range(iter_num+1)]
for i in range(iter_num+1):
    for j in range(1, 552):
        if (i==0):
            dp[i][j][0] = 0
        else:
            for k in range(1, j):
                if (prefix_records[j]==prefix_records[k]):
                    continue
                if (i==1):
                    dp[i][j][0] = LL_mat[1][j]
                    continue
                else:
                    if (dp[i][j][0] < (dp[i-1][k][0] + LL_mat[k][j])):
                        dp[i][j][0] = dp[i-1][k][0] + LL_mat[k][j]
                        dp[i][j][1] = k

In [11]:
# Maximum log likelihood
print(dp[iter_num][551][0])

-4327.507281373537


In [12]:
# Preparing the partitions
k = 551
res = []
while iter_num >= 0:
    res.append(k)
    k = dp[iter_num][k][1]
    iter_num -= 1

res.reverse()

In [13]:
print(res)

[1, 254, 316, 377, 551]


In [14]:
res = res[1:-1] # ignore the 300 (1) and 850 (551) to keep same format as earlier in method 1

In [15]:
print("The 4 optimal buckets are:")
print(f"Score 1 (Lowest Credit Band): 300 to {res[0]+299}")
print(f"Score 2 (Second Lowest Credit Band): {res[0]+300} to {res[1]+299}")
print(f"Score 3 (Second Highest Credit Band): {res[1]+300} to {res[2]+299}")
print(f"Score 4 (Highest Credit Band): {res[2]+300} to 850")

The 4 optimal buckets are:
Score 1 (Lowest Credit Band): 300 to 553
Score 2 (Second Lowest Credit Band): 554 to 615
Score 3 (Second Highest Credit Band): 616 to 676
Score 4 (Highest Credit Band): 677 to 850
